# Shor's algorithm for integer factoring

This example implementation of Shor's algorithm is [adapted from ProjectQ](https://github.com/ProjectQ-Framework/ProjectQ/blob/develop/examples/shor.py).

This self-standing notebook, as a complete work or in any form, (whether dependent on vm6502q/qrack), is licensed under the GNU Lesser General Public License V3.

In [1]:
##############################################################################################
# (C) Daniel Strano and the Qrack contributors 2017-2022. All rights reserved.
#
# "A quantum-inspired Monte Carlo integer factoring algorithm"
#
# This example demonstrates a (Shor's-like) "quantum-inspired" algorithm for integer factoring.
# This approach is similar to Shor's algorithm, except with a uniformly random output from the
# quantum period-finding subroutine. Therefore, we don't need quantum computer simulation for
# this algorithm at all!
#
# (This file was heavily adapted from
# https://github.com/ProjectQ-Framework/ProjectQ/blob/develop/examples/shor.py,
# with thanks to ProjectQ!)
#
# Licensed under the GNU Lesser General Public License V3.
# See https://www.gnu.org/licenses/lgpl-3.0.en.html for details.

In [2]:
import math
import random
from fractions import Fraction

try:
    from math import gcd
except ImportError:
    from fractions import gcd

from pyqrack import QrackSimulator, Pauli

import os
os.environ["QRACK_QUNIT_SEPARABILITY_THRESHOLD"] = "0.001"
os.environ["QRACK_MAX_ALLOC_MB"] = "22528,14336"
os.environ["QRACK_MAX_PAGING_QB"] = "30"
os.environ["QRACK_MAX_PAGE_QB"] = "27"
os.environ["QRACK_MAX_CPU_QB"] = "-1"
os.environ["QRACK_QUNITMULTI_DEVICES"] = "1"

`toFactor` is the number we are factoring. (You may change it.) 

In [3]:
toFactor=2**14-1

The entire algorithm is implemented below. For the first variant, to factor an integer of `n` qubits or less, we have a footprint of `2n`, but PyQrack's `pown()` is limited in qubit width to single GPU "pages," (usually 2 qubits less than maximum for the GPU,) and the operation is "fully entangling" of the internal representation. 

In [4]:
%%time
base = random.randrange(2, toFactor)
if not gcd(base, toFactor) == 1:
    print("Chose non-relative prime, (without need for quantum computing):")
    print("Factor: {}".format(gcd(base, toFactor)))
else:
    qubitCount = math.ceil(math.log2(toFactor))
    sim = QrackSimulator(2 * qubitCount)

    qi = [i for i in range(qubitCount)]
    qo = [(i + qubitCount) for i in range(qubitCount)]

    # run the quantum subroutine
    for i in qi:
        sim.h(i)
    sim.pown(base, toFactor, qi, qo)
    sim.iqft(qi)
    
    b = [Pauli.PauliZ] * qubitCount
    y = sim.measure_pauli(b, qi)
    r = Fraction(y).limit_denominator(toFactor - 1).denominator

    # try to determine the factors
    if r % 2 != 0:
        r *= 2
    apowrhalf = pow(base, r >> 1, toFactor)
    f1 = gcd(apowrhalf + 1, toFactor)
    f2 = gcd(apowrhalf - 1, toFactor)
    fmul = f1 * f2
    if (not fmul == toFactor) and f1 * f2 > 1 and (toFactor // fmul) * fmul == toFactor:
        f1, f2 = f1 * f2, toFactor // (f1 * f2)
    if f1 * f2 == toFactor and f1 > 1 and f2 > 1:
        print("Factors found : {} * {} = {}".format(f1, f2, toFactor))
    else:
        print("Failed: Found {} and {}".format(f1, f2))

Chose non-relative prime, (without need for quantum computing):
Factor: 3
CPU times: user 34 µs, sys: 52 µs, total: 86 µs
Wall time: 76.1 µs


The second implementation requires `2n+2` qubits in footprint to factor a number of `n` qubits or less. However, these methods do not resort to "fully entangled representation," and they are not limited to single GPU "pages."

In [5]:
toFactor=15

In [6]:
def cmul_native(sim, i, a, maxN, qo, qa):
    sim.mcmuln(a, [i], maxN, qo, qa)
    for o in range(len(qa)):
        sim.cswap([i], qa[o], qo[o])
    sim.mcdivn(a, [i], maxN, qo, qa)
    for a in qa:
        sim.m(a)

def phase_root_n(sim, n, q):
    sim.mtrx([1, 0, 0, -1**(1.0 / (1<<(n - 1)))], q)

In [7]:
%%time
# Based on https://arxiv.org/abs/quant-ph/0205095
base = random.randrange(2, toFactor)
factor = gcd(base, toFactor)
if not factor == 1:
    print("Chose non-relative prime, (without need for quantum computing):")
    print("Factors found : {} * {} = {}".format(factor, toFactor // factor, toFactor))
else:
    qubitCount = math.ceil(math.log2(toFactor))
    maxN = 1<<qubitCount
    sim = QrackSimulator(2 * qubitCount + 2, isCpuGpuHybrid=False, isOpenCL=False)

    qo = [i for i in range(qubitCount)]
    qa = [(i + qubitCount) for i in range(qubitCount)]
    qi = 2 * qubitCount
    qm = 2 * qubitCount + 1

    m_results = []

    # Run the quantum subroutine.
    # First, set the multiplication output register to identity, 1.
    sim.x(qo[0])
    for i in range(qubitCount):
        sim.h(qi)
        cmul_native(sim, qi, 1 << i, toFactor, qo, qa)

        # We use the single control qubit "trick" referenced in Beauregard:
        for j in range(len(m_results)):
            if m_results[j]:
                phase_root_n(sim, j + 2, qi)

        m_results.append(sim.m(qi))
        if m_results[-1]:
            sim.x(qi)

    y = 0
    for i in range(len(m_results)):
        if m_results[i]:
            y |= 1<<i
    r = Fraction(y).limit_denominator(toFactor - 1).denominator

    # try to determine the factors
    if r % 2 != 0:
        r *= 2
    apowrhalf = pow(base, r >> 1, toFactor)
    f1 = gcd(apowrhalf + 1, toFactor)
    f2 = gcd(apowrhalf - 1, toFactor)
    fmul = f1 * f2
    if (not fmul == toFactor) and f1 * f2 > 1 and (toFactor // fmul) * fmul == toFactor:
        f1, f2 = f1 * f2, toFactor // (f1 * f2)
    if f1 * f2 == toFactor and f1 > 1 and f2 > 1:
        print("Factors found : {} * {} = {}".format(f1, f2, toFactor))
    else:
        print("Failed: Found {} and {}".format(f1, f2))

std::bad_alloc


RuntimeError: QrackSimulator C++ library raised exception.

Finally, compare all of the above to a "quantum-inspired" Monte Carlo method:

In [8]:
toFactor=17858368935429986533*16978036322460078659
print(toFactor)

303200036445623038476984737418490699247


In [ ]:
%%time
while True:
    base = random.randrange(2, toFactor)
    factor = gcd(base, toFactor)
    if not factor == 1:
        f1, f2 = factor, toFactor // factor
        print("Chose non-relative prime.")
        print("Factors found : {} * {} = {}".format(f1, f2, toFactor))
        break
    else:
        numerator = random.randrange(1, toFactor - 1)
        r = Fraction(numerator).limit_denominator(toFactor - 1).denominator

        # try to determine the factors
        if r % 2 != 0:
            r *= 2
        apowrhalf = pow(base, r >> 1, toFactor)
        f1 = gcd(apowrhalf + 1, toFactor)
        f2 = gcd(apowrhalf - 1, toFactor)
        fmul = f1 * f2
        if (not fmul == toFactor) and f1 * f2 > 1 and (toFactor // fmul) * fmul == toFactor:
            f1, f2 = f1 * f2, toFactor // (f1 * f2)
        if f1 * f2 == toFactor and f1 > 1 and f2 > 1:
            print("Factors found : {} * {} = {}".format(f1, f2, toFactor))
            break
        # else:
        #     print("Failed: Found {} and {}".format(f1, f2))